<a href="https://colab.research.google.com/github/blanchefort/text_mining/blob/master/dynamic_quantization_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Динамическая квантизация BERT
Дообученная модель BERT показывает очень хорошее качество при решении множества NLP-задач. Однако, её не всегда можно применить на практике из-за того, что модель очень большая и работает дастаточно медленно. В связи с этим было придумано несколько способов обойти это ограничение.

Один из способов - метод Dynamic quantization, реализованный в новых версиях Pytorch. Суть метода заключается в том, что мы переводим параметры обученной модели из float в int8. Благодаря этому снижается размер модели и, естественно, уменьшаются вычислительные ресурсы, требуемые для работы модели.

(Как будет видно ниже, для моей модели квантизация существенно уменьшила качество модели, поэтому в моём случае данный метод нельзя использовать. Но, сохраню блокнот как пример квантизации).

### Библиотеки

In [0]:
pip install transformers

In [0]:
import os
import random
from random import randint
import time
import numpy as np
import pandas as pd
import torch
from torch.utils.data import (TensorDataset,
                              DataLoader,
                              RandomSampler)

from keras.preprocessing.sequence import pad_sequences

from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm

SEED = 22
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device.type)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

Using TensorFlow backend.


cpu


## Загрузка обученной модели

In [0]:
# config
config = AutoConfig.from_pretrained('/content/drive/My Drive/colab_data/leroymerlin/model/BERT_model')
# tokenizer
tokenizer = AutoTokenizer.from_pretrained('/content/drive/My Drive/colab_data/leroymerlin/model/BERT_model', pad_to_max_length=True)
# model
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/My Drive/colab_data/leroymerlin/model/BERT_model', config=config)

In [0]:
df = pd.read_csv('/content/drive/My Drive/colab_data/leroymerlin/to_classifier.csv')
sentences = df.name.values
labels = [category_index[i] for i in df.category_1.values]

In [0]:
category_index = {'Водоснабжение': 8,
 'Декор': 12,
 'Инструменты': 4,
 'Краски': 11,
 'Кухни': 15,
 'Напольные покрытия': 5,
 'Окна и двери': 2,
 'Освещение': 13,
 'Плитка': 6,
 'Сад': 9,
 'Сантехника': 7,
 'Скобяные изделия': 10,
 'Столярные изделия': 1,
 'Стройматериалы': 0,
 'Хранение': 14,
 'Электротовары': 3}
category_index_inverted = dict(map(reversed, category_index.items()))

## Проверка работы модели до квантизации

In [0]:
%%time
#model.to(device)
model.eval()

predicts, grounds = [], []
batch = 400

for sku in tqdm(range(batch, len(df), batch)):
    b_s = sentences[sku-batch:sku]
    tokens = [tokenizer.encode(
        sent, 
        add_special_tokens=True, 
        max_length=24, 
        pad_to_max_length='right') for sent in b_s]
    tokens = torch.tensor(tokens).to(device)
    with torch.no_grad():
        outputs = model(tokens)
    logits = outputs[0].detach().cpu().numpy()
    preds = np.argmax(logits, axis=1)
    predicts.extend(preds)
    grounds.extend(labels[sku-batch:sku])


CPU times: user 3h 35min 7s, sys: 1min 21s, total: 3h 36min 28s
Wall time: 1h 48min 43s


In [0]:
print(classification_report(grounds, predicts, target_names=category_index_inverted.values()))

                    precision    recall  f1-score   support

     Водоснабжение       0.94      0.88      0.91     13377
             Декор       1.00      0.40      0.57      2716
       Инструменты       1.00      0.40      0.58       540
            Краски       0.97      0.81      0.88     20397
             Кухни       0.96      0.91      0.93     29920
Напольные покрытия       1.00      0.56      0.72      2555
      Окна и двери       1.00      0.61      0.76      2440
         Освещение       0.98      0.92      0.95     30560
            Плитка       0.97      0.96      0.97     23922
               Сад       0.95      0.98      0.96     49518
        Сантехника       0.97      0.74      0.84     24245
  Скобяные изделия       0.85      0.93      0.89     15280
 Столярные изделия       0.58      0.95      0.72     30329
    Стройматериалы       0.98      0.67      0.80      8532
          Хранение       0.97      0.77      0.86      6237
     Электротовары       0.96      0.88

# Квантизация

In [0]:
%%time
model.to('cpu')
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

CPU times: user 1.41 s, sys: 355 ms, total: 1.77 s
Wall time: 1.5 s


## Проверим размеры модели до и после квантизации

In [0]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [0]:
print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 711.508154
Size (MB): 454.93095


## Проверка работы модели после квантизации

In [0]:
%%time
quantized_model.eval()

predicts, grounds = [], []
batch = 400

for sku in tqdm(range(batch, len(df), batch)):
    b_s = sentences[sku-batch:sku]
    tokens = [tokenizer.encode(
        sent, 
        add_special_tokens=True, 
        max_length=24, 
        pad_to_max_length='right') for sent in b_s]
    tokens = torch.tensor(tokens)
    with torch.no_grad():
        outputs = quantized_model(tokens)
    logits = outputs[0].detach().cpu().numpy()
    preds = np.argmax(logits, axis=1)
    predicts.extend(preds)
    grounds.extend(labels[sku-batch:sku])


CPU times: user 3h 43s, sys: 1min 31s, total: 3h 2min 15s
Wall time: 1h 31min 35s


In [0]:
print(classification_report(grounds, predicts, target_names=category_index_inverted.values()))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

     Водоснабжение       0.66      0.11      0.18     13377
             Декор       0.00      0.00      0.00      2716
       Инструменты       0.00      0.00      0.00       540
            Краски       0.09      0.03      0.05     20397
             Кухни       0.97      0.19      0.31     29920
Напольные покрытия       0.86      0.06      0.11      2555
      Окна и двери       0.77      0.04      0.08      2440
         Освещение       0.97      0.05      0.09     30560
            Плитка       0.89      0.32      0.47     23922
               Сад       0.89      0.14      0.24     49518
        Сантехника       0.72      0.03      0.06     24245
  Скобяные изделия       0.81      0.17      0.28     15280
 Столярные изделия       0.13      0.97      0.23     30329
    Стройматериалы       0.53      0.05      0.08      8532
          Хранение       0.92      0.05      0.10      6237
     Электротовары       0.63      0.09

Итак, мы видим, что после квантизации модель уменьшилась в полтора раза, а скорость предикта на CPU увеличилась лишь на 20%, при этом качество существенно снизилось. Возможно, нужно поэкспериментировать с различными параметрами квантизации. Но пока как пример оставим всё так.

## Сохраним квантизованную модель

In [0]:
!mkdir '/content/drive/My Drive/colab_data/leroymerlin/model/BERT_quntized'

In [0]:
quantized_model.save_pretrained('/content/drive/My Drive/colab_data/leroymerlin/model/BERT_quntized')